# Retrieving Plugins using Plug and Plai API

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/edreisMD/plugnplai/blob/main/examples/retrieve_plugins_api.ipynb)

This example shows how to retrieve compatible plugins using Plug and Plai within LangChain abstractions.

### Install Plug and Plai and necessary LangChain tools

In [1]:
pip install plugnplai -q


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Store all plugins in a dictionary
1. Use the 'get_plugins' function to get all plugins from plugnplai.com
2. Load the specifications and store in a dictionary of key: "name_for_model", value: PluginObject (from plugin.PluginObject - it contains all information to describe and call the plugin later)

In [3]:
import plugnplai as pl

# Get all plugins from plugnplai.com
urls = pl.get_plugins()

dict_plugins = {}

# Lets load 5 plugins as an example 
# (loading all plugins may take a while, we are loading only 50, but feel free to try loading more)
max_line_length = 100
for url in urls[:50]: # 50 plugins - remove [:50] to load all plugins
    try:
        manifest, specification = pl.spec_from_url(url)
        plugin = pl.PluginObject(url, specification, manifest)
        dict_plugins[plugin.name_for_model] = plugin
        # print(f"Plugin {plugin.name_for_model} successfully loaded.", end="", flush=True)
    except Exception as e:
        # print(f"Error loading plugin {url}: {e}", end="", flush=True)
        pass

Http Error: 403 Client Error: Forbidden for url: https://ramp.com/.well-known/ai-plugin.json
Error Connecting: HTTPSConnectionPool(host='theansible.ai', port=443): Max retries exceeded with url: /.well-known/ai-plugin.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f0e565b1130>: Failed to establish a new connection: [Errno -5] No address associated with hostname'))
Http Error: 401 Client Error:  for url: https://staging.valispace.com/rest/?format=openapi
Http Error: 500 Server Error: Internal Server Error for url: https://api.kyuda.io/v1/sdk/user/openapi.json
Error Connecting: HTTPSConnectionPool(host='docketalarm.com', port=443): Max retries exceeded with url: /.well-known/ai-plugin.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f0e56e1fa60>: Failed to establish a new connection: [Errno 101] Network is unreachable'))


In [11]:
#  print number of plugins in dict_plugins
print(f"Number of available plugins: {len(dict_plugins)}")

Number of available plugins: 42


### Retrieve the best plugin for the task
Use pl.retrieve to call PlugnPlai retrieve api based on the user message - https://www.plugnplai.com/_functions/retrieve?text={user_message_here}

In [12]:
dict_plugins.keys()

dict_keys(['speak', 'WolframAlpha', 'Wolfram', 'KlarnaProducts', 'Zapier', 'Shop', 'Milo', 'guru', 'biztoc', 'calculator', 'datasette_datasette_io_3c330f', 'url_reader', 'owd', 'quickchart', 'screenshot', 'portfoliopilot', 'triple_whale_benchmarks', 'recipe_retrieval', 'Semgrep', 'woxo', 'textbelt', 'mrkter', 'kittycad', 'SceneXplain', 'Transvribe', 'web_search', 'messagebird', 'AppyPieAIAppBuilder', 'Asana', 'dalle_plugin', 'MixerBox_OnePlayer_music', 'luma_events', 'safari', 'seoanalysis', 'doctors', 'KalendarAI', 'code_repo_interaction', 'onelook_thesaurus', 'Ambition', 'CensysGPT', 'Slack', 'yt_persona_generator'])

In [13]:
def top_plugins(user_message):

    top_plugins_names = pl.retrieve(text = user_message, available_plugins = dict_plugins.keys())

    #  Convert the top_plugins (list of strings) in a list of PluginObject
    top_plugins = [dict_plugins[plugin] for plugin in top_plugins_names]

    plugins = pl.Plugins(top_plugins)

    # activate the plugins
    for name in top_plugins_names:
        plugins.activate(name)

    return top_plugins_names, plugins

names, plugins = top_plugins("What t shirts are available in klarna?")
print(f"Top plugins: {names}")

Top plugins: ['KlarnaProducts']


# Load the top plugin into the prompt using Plugins class 

In [14]:
print(plugins.prompt)


# SYSTEM MESSAGE
You are a large language model trained to assist humans.
Knowledge Cutoff: 2021-09
Current date: 2023-06-24
Below is a list of available APIs that you can utilize to fulfill user requests. 
When using an API, please follow the specified format to make the API call. 
Don't ask follow-up questions and aim to complete the task with the information provided by the user.

To make an API call, use the following format (using JSON double quotes for the API call parameters):

<API>namespace.operationId({"parameter_name": "parameter_value", ...})</API>

For example, to call an API operation with the operation ID "productsUsingGET" in the "KlarnaProducts" namespace, 
and provide the required parameters "q" and "size", the format would be as follows:

<API>KlarnaProducts.productsUsingGET({"q": "t-shirt", "size": 3})</API>

Please ensure that you use the correct namespace and operation ID, and provide the necessary parameters for each API call. 
After requesting the API, refrain 

# Now lets just follow the example in plugins_step_by_step.ipynb

### Call the LLM using LangChain

In [42]:
# You will need to first define your API key
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from IPython.display import display, Markdown

#### Uncomment or modify the message to test different plugins

In [30]:
# Test Klarna Plugin
HUMAN_MESSAGE = "What t shirts are available in klarna?"

# Test Trip Plugin
# HUMAN_MESSAGE = "I need a hotel in Paris between Dec.3-8"

# Test Speak Plugin
# HUMAN_MESSAGE = "How to say I love you in Portuguese?"

### Retrieve plugins

In [37]:
names, plugins = top_plugins(HUMAN_MESSAGE)

#### Call LLM

In [38]:
chat = ChatOpenAI(temperature=0, model="gpt-4-0613")

messages = [
    SystemMessage(content=plugins.prompt),
    HumanMessage(content=HUMAN_MESSAGE)
]

res = chat(messages)

llm_first_response = res.content

print(llm_first_response)

<API>KlarnaProducts.productsUsingGET({"countryCode": "us", "q": "t-shirt", "size": 5})</API>


## Parse the LLM response

In [39]:
# import the parser function
from plugnplai import parse_llm_response

# Parse the LLM response importing '
call_dict = parse_llm_response(llm_first_response)
print(call_dict)

{'plugin_name': 'KlarnaProducts', 'operation_id': 'productsUsingGET', 'parameters': {'countryCode': 'us', 'q': 't-shirt', 'size': 5}}


## Call Plugin

In [40]:
r = plugins.call_api(plugin_name = call_dict['plugin_name'],
                    operation_id = call_dict['operation_id'],
                    parameters = call_dict['parameters']
                    )

print(r.status_code)
api_response = r.json()
r.json()

200


{'products': [{'name': 'Polo Ralph Lauren Slim Fit Cotton T-shirt 3-pack',
   'url': 'https://www.klarna.com/us/shopping/pl/cl10001/3201838628/Clothing/Polo-Ralph-Lauren-Slim-Fit-Cotton-T-shirt-3-pack/?utm_source=openai&ref-site=openai_plugin',
   'price': '$27.65',
   'attributes': ['Material:Cotton',
    'Target Group:Man',
    'Color:Gray,White,Blue,Multicolor,Black',
    'Size:S,XL,XS,L,M,XXL']},
  {'name': 'Palm Angels Bear T-shirt - White',
   'url': 'https://www.klarna.com/us/shopping/pl/cl10001/3200094728/Clothing/Palm-Angels-Bear-T-shirt-White/?utm_source=openai&ref-site=openai_plugin',
   'price': '$218.00',
   'attributes': ['Size (US):12,14,8,10',
    'Material:Cotton',
    'Target Group:Man',
    'Color:White',
    'Size:S,XL,3XL,XS,L,M,XXL']},
  {'name': 'adidas Adicolor Classics Trefoil T-shirt - Black/White',
   'url': 'https://www.klarna.com/us/shopping/pl/cl10001/3201095924/Clothing/adidas-Adicolor-Classics-Trefoil-T-shirt-Black-White/?utm_source=openai&ref-site=opena

## LLM responds using the API data

In [41]:
from plugnplai import Plugins

api_return_prompt = f"""
Assistant is a large language model with access to plugins.

Assistant called a plugin in response to this human message:
# HUMAN MESSAGE
{HUMAN_MESSAGE}

# API REQUEST SUMMARY
{llm_first_response}

# API RESPONSE
{api_response}
"""

# # Install the plugins ewith the original template
# plugins = Plugins.install_and_activate(urls)

chat = ChatOpenAI(temperature=0, model="gpt-4-0613")
# chat = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

messages = [
    SystemMessage(content=api_return_prompt),
    HumanMessage(content="HUMAN_MESSAGE")
]

res = chat(messages)

display(Markdown(res.content))

Here are some t-shirts available on Klarna:

1. [Polo Ralph Lauren Slim Fit Cotton T-shirt 3-pack](https://www.klarna.com/us/shopping/pl/cl10001/3201838628/Clothing/Polo-Ralph-Lauren-Slim-Fit-Cotton-T-shirt-3-pack/?utm_source=openai&ref-site=openai_plugin) - $27.65
   - Material: Cotton
   - Target Group: Man
   - Color: Gray, White, Blue, Multicolor, Black
   - Size: S, XL, XS, L, M, XXL

2. [Palm Angels Bear T-shirt - White](https://www.klarna.com/us/shopping/pl/cl10001/3200094728/Clothing/Palm-Angels-Bear-T-shirt-White/?utm_source=openai&ref-site=openai_plugin) - $218.00
   - Size (US): 12, 14, 8, 10
   - Material: Cotton
   - Target Group: Man
   - Color: White
   - Size: S, XL, 3XL, XS, L, M, XXL

3. [adidas Adicolor Classics Trefoil T-shirt - Black/White](https://www.klarna.com/us/shopping/pl/cl10001/3201095924/Clothing/adidas-Adicolor-Classics-Trefoil-T-shirt-Black-White/?utm_source=openai&ref-site=openai_plugin) - $11.22
   - Material: Cotton
   - Target Group: Man
   - Color: Black
   - Size: S, XL, XS, L, M, XXL

4. [Psycho Bunny Mens Copa Gradient Logo Graphic Tee](https://www.klarna.com/us/shopping/pl/cl10001/3203663222/Clothing/Psycho-Bunny-Mens-Copa-Gradient-Logo-Graphic-Tee/?utm_source=openai&ref-site=openai_plugin) - $49.00
   - Material: Cotton
   - Target Group: Man
   - Color: White, Blue, Black, Orange
   - Size: XXS, S, XL, 3XL, XS, L, M, XXL

5. [Armani Exchange Men's Script Logo T-shirt](https://www.klarna.com/us/shopping/pl/cl10001/3202187360/Clothing/Armani-Exchange-Men-s-Script-Logo-T-shirt/?utm_source=openai&ref-site=openai_plugin) - $36.95
   - Material: Cotton
   - Target Group: Man
   - Color: Blue, Black
   - Size: S, XL, XS, L, M, XXL

Please note that prices and availability are subject to change.